In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Hybrid import *
model = ScoresHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2568 (20.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [ ]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Hybrid import LinearHybridRecommender, LinearZScoreNormalizedHybridRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

model = LinearHybridRecommender
slim = SLIMElasticNetRecommender(URM_train)
rp3 = RP3betaRecommender(URM_train)
itemknn = ItemKNNCFRecommender(URM_train)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)
rp3.fit(topK=32, alpha=0.22028996694465927, beta=0.17339063177346345)
#rp3.fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

SLIMElasticNetRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 329 ( 1.5%) items with no interactions.
RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 329 ( 1.5%) items with no interactions.


In [14]:
import optuna as op
def objective(trial):
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    beta = trial.suggest_float("beta", 0.0, 1.0 - alpha)
    
    recommender = ScoresHybridRecommender(URM_train, slim, rp3, itemknn)
    recommender.fit(alpha=alpha, beta=beta)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [15]:
best_params = {
    "alphas": [1, 1, 1],
}

study_name = "hybrid-scores-linear-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)h

/tmp/ipykernel_56596/2307385466.py:7: FutureWarning: delete_study() got {'study_name', 'storage'} as positional arguments but they were expected to be given as keyword arguments.
  op.delete_study(study_name, storage_name)
[I 2023-12-16 00:31:34,787] A new study created in RDB with name: hybrid-scores-linear-study


ScoresHybridRecommender: URM Detected 463 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 350 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10065 (100.0%) in 8.09 sec. Users per second: 1244


[I 2023-12-16 00:31:42,981] Trial 0 finished with value: 0.03208112349095981 and parameters: {'alpha': 0.5740185614743544, 'beta': 0.11697573697529057}. Best is trial 0 with value: 0.03208112349095981.


ScoresHybridRecommender: URM Detected 463 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 350 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10065 (100.0%) in 7.97 sec. Users per second: 1262


[I 2023-12-16 00:31:51,009] Trial 1 finished with value: 0.031881007577729124 and parameters: {'alpha': 0.4521369355974042, 'beta': 0.12554538518073804}. Best is trial 0 with value: 0.03208112349095981.


...

[I 2023-12-16 00:44:31,554] Trial 97 finished with value: 0.032223610026888956 and parameters: {'alpha': 0.4990082098008028, 'beta': 0.35640267026553496}. Best is trial 93 with value: 0.03232528643184406.


ScoresHybridRecommender: URM Detected 463 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 350 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10065 (100.0%) in 7.91 sec. Users per second: 1272


[I 2023-12-16 00:44:39,581] Trial 98 finished with value: 0.03227508496360971 and parameters: {'alpha': 0.45586395956051734, 'beta': 0.3329198895865545}. Best is trial 93 with value: 0.03232528643184406.


ScoresHybridRecommender: URM Detected 463 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 350 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 10065 (100.0%) in 7.68 sec. Users per second: 1310


[I 2023-12-16 00:44:47,326] Trial 99 finished with value: 0.032203585424897166 and parameters: {'alpha': 0.5312935610316081, 'beta': 0.373681817826226}. Best is trial 93 with value: 0.03232528643184406.


In [16]:
study.best_params

{'alpha': 0.5072513954300486, 'beta': 0.30902582824263064}

In [17]:
final = ScoresHybridRecommender(URM_train_validation, slim, rp3, itemknn)
final.fit(**study.best_params)

ScoresHybridRecommender: URM Detected 219 ( 1.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 132 ( 0.6%) items with no interactions.


In [18]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2184 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10454 (100.0%) in 7.87 sec. Users per second: 1328


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.089047                 0.151877  0.127205  0.045314    0.076721   
 
              MRR     NDCG       F1  HIT_RATE ARHR_ALL_HITS  ... COVERAGE_USER  \
 cutoff                                                      ...                 
 10      0.256018  0.13966  0.10476  0.495313      0.336462  ...      0.827188   
 
        COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI SHANNON_ENTROPY  \
 cutoff                                                                
 10              0.409717    0.827188       0.072895       10.533339   
 
        RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY  \
 cutoff                                                                         
 10                       0.998302             0.209025              0.811384   
 
        RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cut

In [19]:
evaluator_test.evaluateRecommender(slim)

EvaluatorHoldout: Processed 10454 (100.0%) in 6.03 sec. Users per second: 1734


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.078955                 0.137148  0.116498  0.037969    0.066754   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.231649  0.124553  0.094121  0.474077      0.294289  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827188          0.392151    0.827188       0.050939   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.043526                   0.997462             0.147806   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   